# Pcap analysis

In [2]:
import pandas as pd
from scapy.all import *
from scapy.contrib.coap import *
from collections import Counter

In [3]:
packets = rdpcap('capture.pcapng')

CQ1) How many different Confirmable PUT requests obtained an 
unsuccessful response from the local CoAP server?

In [56]:
# Select Confirmable PUT request
put_con_packets = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code == 3 and pkt[CoAP].type == 0]

# Select only requests with unsuccessful responce from the local CoAP server
uns_resp = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code > 69 and pkt["IP"].src == "127.0.0.1"]

count = len({pkt[CoAP].token for pkt in put_con_packets} & {pkt[CoAP].token for pkt in uns_resp})

print("Answer:")
print(count)

Answer:
22


CQ2) How many CoAP resources in the coap.me public server received the 
same number of unique Confirmable and Non Confirmable GET requests?

Assuming a resource receives X different CONFIRMABLE requests and Y different 
NONCONFIRMABLE  GET requests, how many resources have X=Y, with X>0?

In [75]:
con_resources = Counter()
non_resources = Counter()

for pkt in packets: 
    if pkt.haslayer(CoAP) and pkt[CoAP].code == 1 and pkt["IP"].dst == "134.102.218.18":
        uri_path_str = ""
        for option in pkt[CoAP].options:
            if option[0] == "Uri-Path":
                uri_path = option[1].decode('utf-8')
                if uri_path:
                    uri_path_str = uri_path_str + "/" + uri_path
        if pkt[CoAP].type == 0:    
            con_resources[uri_path_str]+=1
        else:
            non_resources[uri_path_str]+=1
            

result = [r for r in (set(con_resources.keys()) & set(non_resources.keys())) if con_resources[r] == non_resources[r]]

count = len(result)

print(result)

print("Answer:")
print(count)

['/validate', '/secret', '/large']
Answer:
3
